In [1]:
import psycopg2
import echonestkeys as secret
import json
import urllib2
import time 
import requests

# import pandas as pd
# import numpy as np
# from collections import Counter

from fuzzywuzzy import fuzz, process
# from nltk.tag import pos_tag
# from nltk.tokenize import word_tokenize
# from nltk.tokenize import sent_tokenize

# from scraper import country_adjectives
# from scraper import putumayo_artists
from scraper import artists_wikipedia
echo_limit = 0
def rate_limiter():
    global echo_limit
    echo_limit += 1
    if echo_limit%119 == 0:
        time.sleep(63)
        
def rate_reset():
    global echo_limit
    echo_limit = 0
    
def get_json(url):
    response = requests.get(url)
    if response.status_code==200:
        return json.loads(response.text)
#     print 'Response code not 200 for %s' % url
    return None

def fuzzy_match(a, names):
    lower = unicode.lower
    strlower = str.lower
    if a == names[0]:
        return 0
    try:
        if strlower(a) == lower(names[0]):
            return 0
    except:
        if lower(a) == lower(names[0]):
            return 0
    best = max([(fuzz.ratio(a, n),n) for n in names])
    if best[0]>0.7:
        return names.index(best[1])
    else:
        return -1
 

In [2]:
   
def get_sp_artist(query):
    q = '+'.join(query.split()).strip()
    url = "https://api.spotify.com/v1/search?q=%s&type=artist" % q
    results = get_json(url)['artists']['items']
    if results != []:
        names = [r['name'] for  r in results]
        index = fuzzy_match(query, names)
        if index != -1:
            return results[index]
    return None
    
def sp_artist(query):   
    artist = get_sp_artist(query)
    if artist:
        sp_artistId = artist['id']
        sp_artistName = artist['name']
        sp_artistGenres = artist['genres']
        sp_artistPopularity = artist['popularity']
        sp_artistImageLinks = artist['images']
        return (sp_artistId, sp_artistName)
    return None


def get_ec_track(artistId, track, name):
    url = 'http://developer.echonest.com/api/v4/song/search?api_key=%s&format=json&results=1\
&artist_id=spotify:artist:%s&title=%s&bucket=audio_summary&bucket=artist_location' % (secret.apikey(), artistId , track)
    rate_limiter()
    response = get_json(url)
    if response:
        response = response['response']
        status = response['status']['code']
        if status == 0:
            songs = response['songs']
            if songs!=[]:
                return songs[0]
    return None

def get_sp_topTracks(sp_artistId, sp_artistName):
    url = 'https://api.spotify.com/v1/artists/%s/top-tracks?country=US' % sp_artistId
    response = get_json(url)
    if response:
        tracks = response['tracks']
        if tracks:
            for track in tracks:
                sp_trackId = track['id']
                sp_trackName = track['name']
                sp_trackPreview = track['preview_url']
                sp_trackExplicit = track['explicit']
                sp_trackArtists = [(a['id'],a['name']) for a in track['artists']]
                sp_trackDuration = track['duration_ms']
                q_trackName = '%20'.join(sp_trackName.split(' '))
                ec_track = get_ec_track(sp_artistId, q_trackName, sp_artistName)
                if ec_track:
                    ec_trackId = ec_track['id']
                    audsum = ec_track['audio_summary']
                    ec_audioSummary = {s: audsum[s] for s in audsum if s != 'analysis_url'}
                    ec_artistId = ec_track['artist_id']
                    ec_artistLocation = ec_track['artist_location']
                    if ec_artistLocation != []:
                        print 'No location for %s' % sp_artistName
                    return
        else:
            print 'No top tracks: %s, %s' % (sp_artistName, url)
    else:
        print 'No response: %s, %s' % (sp_artistName, url)
    return

In [3]:
# rate_reset()
artists2013 = artists_wikipedia(2013)
for artist in artists2013:
    rep = sp_artist(artist)
    if rep:
        spid = rep[0]
        spname = rep[1]
        get_sp_topTracks(spid, spname)

No response: 2 Chainz - We%20Own%20It%20(Fast%20&%20Furious), http://developer.echonest.com/api/v4/song/search?api_key=0GGBLPK4OHMQIJYQ2&format=json&results=1&artist_id=spotify:artist:17lzZA2AlOHwCwFALHttmp&title=We%20Own%20It%20(Fast%20&%20Furious)&bucket=audio_summary&bucket=artist_location
No top tracks: 98, https://api.spotify.com/v1/artists/3XxuvxvLFTTRKlEWE3fAU8/top-tracks?country=US
No top tracks: Andy Kaufman, https://api.spotify.com/v1/artists/0gLaNyHiFTc6SwZY2Cd7LG/top-tracks?country=US
No top tracks: Angel, https://api.spotify.com/v1/artists/7FjElQ2FxgAbDGSrcE7K6O/top-tracks?country=US
songs[]:  Atoms For Peace - Tamer%20Animals%20-%20Atoms%20For%20Peace%20Remix, http://developer.echonest.com/api/v4/song/search?api_key=0GGBLPK4OHMQIJYQ2&format=json&results=1&artist_id=spotify:artist:7tA9Eeeb68kkiG9Nrvuzmi&title=Tamer%20Animals%20-%20Atoms%20For%20Peace%20Remix&bucket=audio_summary&bucket=artist_location
No response: Attila - #Collegeparty, http://developer.echonest.com/api/v

In [40]:

try:
    conn = psycopg2.connect("dbname='template1' user='dbuser' host='localhost' password='dbpass'")
except:
    print "I am unable to connect to the database"

I am unable to connect to the database


In [7]:
# rate_reset()
from scraper import putumayo_artists
url = 'https://www.putumayo.com/african-blues-9-7-15-9-13-15/'
artists = putumayo_artists(url, 5)
artists.keys()
for artist in artists.keys():
    rep = sp_artist(artist)
    if rep:
        spid = rep[0]
        spname = rep[1]
        get_sp_topTracks(spid, spname)

No response: The Bilz - 2step%20Bhangra%20(feat.%20so%20D%20&%20Kashif), http://developer.echonest.com/api/v4/song/search?api_key=0GGBLPK4OHMQIJYQ2&format=json&results=1&artist_id=spotify:artist:2MJUpOL9aXlJPJx2qmwVRB&title=2step%20Bhangra%20(feat.%20so%20D%20&%20Kashif)&bucket=audio_summary&bucket=artist_location
No response: The Bilz - O%20Meri%20Rani%20(Spanish%20Fly)%20[feat.%20Kashif%20&%20Drega], http://developer.echonest.com/api/v4/song/search?api_key=0GGBLPK4OHMQIJYQ2&format=json&results=1&artist_id=spotify:artist:2MJUpOL9aXlJPJx2qmwVRB&title=O%20Meri%20Rani%20(Spanish%20Fly)%20[feat.%20Kashif%20&%20Drega]&bucket=audio_summary&bucket=artist_location
No response: The Bilz - Mere%20Khyalon%20Mein%20(feat.%20Master%20D%20&%20Kashif), http://developer.echonest.com/api/v4/song/search?api_key=0GGBLPK4OHMQIJYQ2&format=json&results=1&artist_id=spotify:artist:2MJUpOL9aXlJPJx2qmwVRB&title=Mere%20Khyalon%20Mein%20(feat.%20Master%20D%20&%20Kashif)&bucket=audio_summary&bucket=artist_locatio

In [1]:
[1,2] + [2,3]

[1, 2, 2, 3]